In [1]:
###### Ce code est a executer sur Google Coolab car mon pc ne dispose pas d'assez de ressource ###########33


import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score

In [2]:
import tensorflow as tf
import transformers

print("Version de TensorFlow :", tf.__version__)
print("Version de Transformers :", transformers.__version__)

Version de TensorFlow : 2.12.0
Version de Transformers : 4.37.1


In [3]:
# Chargement du jeu de données
data = pd.read_csv("data/BenignAndMaliciousDataset.csv")

### Les features importants ont deja ete selectionner avec l'algo de Gain d'information et de Gain de ratio et coefficient de pearson
X = data[['DNSRecordType', 'NumericSequence', 'NumericRatio', 'StrangeCharacters', 'ConsoantRatio', 'RegisteredOrg']]
y = data['Class']


In [4]:
# Prétraitement des fonctionnalités numériques
numeric_features = data[['NumericSequence', 'NumericRatio', 'StrangeCharacters', 'ConsoantRatio', 'SubdomainNumber']]
scaler = MinMaxScaler()
scaled_numeric_features = scaler.fit_transform(numeric_features)

# Tokenisation des attributs catégoriels avec BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

categorical_feature1 = data['DNSRecordType'].astype(str).tolist()
categorical_feature2 = data['RegisteredOrg'].astype(str).tolist()

combined_text = [f"{feat1} [SEP] {feat2}" for feat1, feat2 in zip(categorical_feature1, categorical_feature2)]
tokens = tokenizer(combined_text, padding=True, truncation=True, return_tensors='tf', max_length=64)


In [5]:
# Entrées
input_num = layers.Input(shape=(scaled_numeric_features.shape[1],))
input_text = layers.Input(shape=(tokens['input_ids'].shape[1],), dtype=tf.int32)


In [6]:
# Branches du modèle
# Branche numérique - FFN
num_branch = layers.Dense(64, activation='relu')(input_num)
num_branch = layers.Dense(32, activation='relu')(num_branch)


In [7]:
# Branche textuelle - BERT
bert_model = TFBertModel.from_pretrained('bert-base-uncased')
text_branch = bert_model(input_text)['pooler_output']

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [8]:
 #Fusion des branches
merged = layers.concatenate([num_branch, text_branch])

# Couches supplémentaires après la fusion
merged = layers.Dropout(0.5)(merged)
merged = layers.Dense(16, activation='relu')(merged)
output = layers.Dense(1, activation='sigmoid')(merged)

# Création et compilation du modèle
model = Model(inputs=[input_num, input_text], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [10]:
# Diviser les données en ensembles d'entraînement et de test
text_train, text_test, num_train, num_test, labels_train, labels_test = train_test_split(
    tokens['input_ids'].numpy(),  # Convertir les tenseurs TensorFlow en tableaux NumPy
    scaled_numeric_features,
    y,
    test_size=0.2,
    random_state=42
)

# Entraînement du modèle
history = model.fit([num_train, text_train], labels_train, epochs=2, batch_size=64, validation_data=([num_test, text_test], labels_test))


Epoch 1/2
 254/1125 [=====>........................] - ETA: 27:07:56 - loss: 0.6139 - accuracy: 0.6768

In [ ]:
# Évaluation du modèle
y_pred = model.predict([num_test, text_test])
y_pred = np.round(y_pred)
evaluation_results = model.evaluate([num_test, text_test], labels_test)
print("Perte sur les données de test:", evaluation_results[0])
print("Précision sur les données de test:", evaluation_results[1])

# Affichage des métriques
loss = history.history['loss']
accuracy = history.history['accuracy']
val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']


In [ ]:
# Affichage des courbes
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(range(1, len(loss) + 1), loss, 'b', label='Perte d\'entraînement')
plt.plot(range(1, len(val_loss) + 1), val_loss, 'r', label='Perte de validation')
plt.title('Courbe de Perte')
plt.xlabel('Épochs')
plt.ylabel('Perte')
plt.legend()

In [ ]:
plt.subplot(1, 2, 2)
plt.plot(range(1, len(accuracy) + 1), accuracy, 'g', label='Exactitude d\'entraînement')
plt.plot(range(1, len(val_accuracy) + 1), val_accuracy, 'r', label='Exactitude de validation')
plt.title('Courbe d\'Exactitude')
plt.xlabel('Épochs')
plt.ylabel('Exactitude')
plt.legend()
plt.show()

In [ ]:
# Matrice de confusion
conf_matrix = confusion_matrix(labels_test, y_pred)
df_cm = pd.DataFrame(conf_matrix, index=['Actual 0', 'Actual 1'], columns=['Predicted 0', 'Predicted 1'])
sn.heatmap(df_cm, annot=True, cmap='Blues', fmt='g')
plt.title('Matrice de Confusion')
plt.show()


In [ ]:
# Courbe ROC
fpr, tpr, _ = roc_curve(labels_test, y_pred)
plt.plot(fpr, tpr, label='Courbe ROC')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('Taux de faux positifs (FPR)')
plt.ylabel('Taux de vrais positifs (TPR)')
plt.title('Courbe ROC')
plt.legend()
plt.show()

# Aire sous la courbe (AUC)
auc = roc_auc_score(labels_test, y_pred)
print('Aire sous la courbe (AUC) :', auc)